In [1]:
from junifer.storage import HDF5FeatureStorage
from julearn.api import run_cross_validation
from julearn.pipeline import PipelineCreator
from julearn.viz import plot_scores
from julearn.stats.corrected_ttest import corrected_ttest
import pandas as pd
import seaborn as sns
from sklearn.svm import LinearSVC
import warnings


It is highly recommended to configure Git before using DataLad. Set both 'user.name' and 'user.email' configuration variables.


In [2]:
storage = HDF5FeatureStorage(uri='./data/AOMIC_Parcels_sch100x7.hdf5')

In [22]:
df_parcellations = storage.read_df('VBM_GM_Schaefer100x7_mean_aggregation')
df_histograms= storage.read_df('VBM_GM_Histogram_100bins_IXI_hist')
df_demographics = pd.read_csv('./data/participants.tsv',sep='\t')
df_demographics.rename(columns={"participant_id": "subject"}, inplace=True)

In [23]:
df_parcellations.dropna(inplace=True)
df_histograms.dropna(inplace=True)


In [24]:
df_parcellations.columns = df_parcellations.columns.astype(str)
df_histograms.columns = df_histograms.columns.astype(str)

# List of columns for each dataframe
X_parcellations = list(df_parcellations.columns)
X_histograms = list(df_histograms.columns)

# Merge with df_demographics on 'subject'
df_full_parcellations = df_parcellations.merge(df_demographics, on="subject")
df_full_histograms = df_histograms.merge(df_demographics, on="subject")


In [25]:
creator = PipelineCreator(problem_type="classification")
creator.add("zscore")
creator.add(
    "svm",
    C=(0.001, 100, "log-uniform"),
)

In [30]:
search_params = {
    "kind": "optuna",
    "cv":4
}

scoring = ["balanced_accuracy", "accuracy"]

scores_hists, model_hists, inspector_hists = run_cross_validation(
    X=X_histograms,
    y='sex',
    data=df_full_histograms,
    search_params=search_params,
    model=creator,
    return_train_score=True,
    return_inspector=True,
    cv=4,
    scoring = scoring,

)


scores_schaefer, model_schaefer, inspector_schaefer = run_cross_validation(
    X=X_parcellations,
    y='sex',
    data=df_full_parcellations,
    search_params=search_params,
    model=creator,
    return_train_score=True,
    return_inspector=True,
    cv=4,
    scoring = scoring,
)

/home/hsreekri/Hsreekri/lib/python3.11/site-packages/julearn/prepare.py:505: RuntimeWarning: The following columns are not defined in X_types: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99']. They will be treated as continuous.
  warn_with_log(

/home/hsreekri/Hsreekri/lib/python3.11/site-packages/julearn/pipeline/pipeline_creator.py:1041: ExperimentalWarning: OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the futur

In [31]:
scores_schaefer

,fit_time,score_time,estimator,test_balanced_accuracy,train_balanced_accuracy,test_accuracy,train_accuracy,n_train,n_test,repeat,fold,cv_mdsum
0,0.839818,0.008981,"OptunaSearchCV(cv=KFold(n_splits=4, random_sta...",0.840558,0.964941,0.836283,0.964497,676,226,0,0,bc7087515161a73a5a6aff57863f3803
1,0.811612,0.009305,"OptunaSearchCV(cv=KFold(n_splits=4, random_sta...",0.809943,0.995650,0.805310,0.995562,676,226,0,1,bc7087515161a73a5a6aff57863f3803
2,0.883196,0.009518,"OptunaSearchCV(cv=KFold(n_splits=4, random_sta...",0.844477,0.935275,0.840000,0.935007,677,225,0,2,bc7087515161a73a5a6aff57863f3803
3,0.817973,0.009235,"OptunaSearchCV(cv=KFold(n_splits=4, random_sta...",0.799818,0.915409,0.800000,0.914328,677,225,0,3,bc7087515161a73a5a6aff57863f3803


In [32]:
scores_hists

,fit_time,score_time,estimator,test_balanced_accuracy,train_balanced_accuracy,test_accuracy,train_accuracy,n_train,n_test,repeat,fold,cv_mdsum
0,0.823943,0.009692,"OptunaSearchCV(cv=KFold(n_splits=4, random_sta...",0.758276,0.792859,0.756637,0.794379,676,226,0,0,bc7087515161a73a5a6aff57863f3803
1,0.818658,0.009120,"OptunaSearchCV(cv=KFold(n_splits=4, random_sta...",0.768868,0.824863,0.761062,0.828402,676,226,0,1,bc7087515161a73a5a6aff57863f3803
2,0.925513,0.009872,"OptunaSearchCV(cv=KFold(n_splits=4, random_sta...",0.738978,0.789465,0.751111,0.790251,677,225,0,2,bc7087515161a73a5a6aff57863f3803
3,0.855256,0.009300,"OptunaSearchCV(cv=KFold(n_splits=4, random_sta...",0.679678,0.824127,0.680000,0.827179,677,225,0,3,bc7087515161a73a5a6aff57863f3803


In [33]:
scores_hists['model'] = 'AOMIC_Histograms'
scores_schaefer['model'] = 'AOMIC_Schaefer'
plot_scores(scores_schaefer,scores_hists)


BokehModel(combine_events=True, render_bundle={'docs_json': {'e36c1753-fb8e-4be3-8ebd-18159853c0a8': {'version…

/home/hsreekri/Hsreekri/lib/python3.11/site-packages/julearn/stats/corrected_ttest.py:175: RuntimeWarning: The training set sizes are not the same. Will use a rounded average.
  warn_with_log(

/home/hsreekri/Hsreekri/lib/python3.11/site-packages/julearn/stats/corrected_ttest.py:184: RuntimeWarning: The testing set sizes are not the same. Will use a rounded average.
  warn_with_log(

